In [ ]:
def Tensor_extended_global_process(A, L1, U1, B, Bmt, m):
    a = B.shape
    J1, J2, K1, K2 = a
    I4 = Identity4(K1, K2)
    H = sp.lil_matrix((2 * (m + 1), 2 * m))
    T = sp.lil_matrix((2 * (m + 1), 2 * m))
    K22 = 2 * K2
    V = np.zeros((J1, J2, K1, (m + 1) * K22))
    Vold = np.zeros((J1, J2, K1, K22))

    VV = np.linalg.solve(U1, np.dot(L1, Bmt))
    VV = VV.reshape((J1, J2, K1, K2))
    Vold[..., K2:K22] = VV
    Vold[..., :K2] = B
    Vj, Omega = global_ortho(Vold, K2)
    o11 = Omega[0, 0]
    o22inv = 1 / Omega[1, 1]
    o12 = Omega[0, 1]
    V[..., :K22] = Vj

    k = 1
    for j in range(1, m + 1):
        W = np.zeros((J1, J2, K1, K22))
        jj = slice(2 * j - 1, 2 * j + 1)
        j1 = slice(0, K2)
        j2 = slice(K2, K22)
        W[..., j1] = mode_dot(A, Vj[..., j1], (2, 3))

        matVj2 = Vj[..., j2].reshape(J1 * J2, K1 * K2)
        VV = np.linalg.solve(U1, np.dot(L1, matVj2))
        W[..., j2] = VV.reshape(J1, J2, K1, K2)

        for i in range(1, j + 1):
            ii = slice(2 * i - 1, 2 * i + 1)
            i1 = slice((i - 1) * K22, i * K22)
            H[ii, jj] = DiamPD(V[..., i1], W, 2, K2)
            dd = KronTens(H[ii, jj].toarray(), I4)
            W -= mode_dot(V[..., i1], dd, (2, 3))

        Vj, H[jj.stop:jj.stop + 2, jj] = global_ortho(W, K2)
        jnew = slice(j * K22, (j + 1) * K22)
        V[..., jnew] = Vj

        Ijj = sp.eye(2 * (j + 1))
        if j == 1:
            T[0:4, 0] = H[0:4, 0]
            T[0:4, 1] = o22inv * (o11 * Ijj[:, 0] - o12 * H[0:4, 0])
        else:
            Ijjk = sp.eye(2 * (k + 1))
            T[0:2 * (j + 1), 2 * j - 1] = H[0:2 * (j + 1), 2 * j - 1]
            aa = H[2 * k + 2, 2 * k]
            aa1 = 1 / aa
            T[0:2 * (j + 1), 2 * k + 2] = aa1 * (Ijj[:, 2 * k] - T[0:2 * (j + 1), :2 * k + 1] @ H[:2 * k + 1, 2 * k])
            k += 1

    return V, H, o11, T